# 文生图-Dreambooth

同济子豪兄 2023-6-13

https://github.com/open-mmlab/mmagic/tree/main/configs

## 进入 MMagic 主目录

In [ ]:
import os
os.chdir('mmagic')

## 在数据集上训练Dreambooth

In [ ]:
# !bash tools/dist_train.sh configs/dreambooth/dreambooth.py 1

!bash tools/dist_train.sh configs/dreambooth/dreambooth-lora.py 1

## 用训练好的模型做预测

In [ ]:
from mmengine import Config

from mmagic.registry import MODELS
from mmagic.utils import register_all_modules

register_all_modules()

In [ ]:
!ls
cfg = Config.fromfile('configs/dreambooth/dreambooth-lora.py')

In [ ]:
dreambooth_lora = MODELS.build(cfg.model)

In [ ]:
state = torch.load('mmagic/work_dirs/dreambooth-lora/iter_1000.pth')['state_dict']

In [ ]:
def convert_state_dict(state):
    state_dict_new = {}
    for k, v in state.items():
        if '.module' in k:
            k_new = k.replace('.module', '')
        else:
            k_new = k
        if 'vae' in k:
            if 'to_q' in k:
                k_new = k.replace('to_q', 'query')
            elif 'to_k' in k:
                k_new = k.replace('to_k', 'key')
            elif 'to_v' in k:
                k_new = k.replace('to_v', 'value')
            elif 'to_out' in k:
                k_new = k.replace('to_out.0', 'proj_attn')
        state_dict_new[k_new] = v
    return state_dict_new

In [ ]:
dreambooth_lora.load_state_dict(convert_state_dict(state))
dreambooth_lora = dreambooth_lora.cuda()

In [ ]:
samples = dreambooth_lora.infer('side view of sks dog', guidance_scale=5)

In [ ]:
samples['samples'][0]

In [ ]:
samples = dreambooth_lora.infer('ear close-up of sks dog', guidance_scale=5)

In [ ]:
samples['samples'][0]